In [1]:
import pandas as pd
import gensim.downloader as api
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.decomposition import PCA
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
wv_google_300d = api.load('word2vec-google-news-300')
wv_twitter_50d = api.load('glove-twitter-50')


## Cos similarity

In [178]:
history_df = pd.read_csv('history_predictions.csv')

incorrect_predictions = history_df.loc[history_df['correct'] == 0]
correct_predictions = history_df.loc[history_df['correct'] == 1]

stopword_list = stopwords.words('english')

stopword_list.append("-B-")
stopword_list.append("-A-")
stopword_list.append("<unk>")

stemmer = nltk.stem.porter.PorterStemmer()

incorrect_predictions['decoded_segment']

9       ['-B-', 'okay', '.', 'i', "'m", 'done', 'anymo...
23      ['-B-', 'i', 'have', 'the', 'same', 'picture',...
60      ['-B-', 'guy', 'on', 'white', 'bike', '-A-', '...
64      ['-B-', 'do', 'you', 'have', 'the', 'lady', 's...
71      ['-B-', 'kk', 'little', 'girl', 'helping', 'da...
                              ...                        
2548    ['-B-', 'teenager', 'with', 'child', '-A-', 'b...
2553     ['-B-', 'plaid', 'apron', '-A-', '<unk>', 'guy']
2560    ['-A-', 'i', 'have', 'a', 'woman', 'with', 'a'...
2571    ['-A-', 'black', 'shirt', 'red', 'lettering', ...
2579    ['-B-', 'laptop', 'on', 'black', 'stand', 'and...
Name: decoded_segment, Length: 263, dtype: object

In [238]:
vectors = []

# incorrect_predictions['decoded_segment'], decoded

wrong_pred = incorrect_predictions['decoded_segment']
good_pred = correct_predictions['decoded_segment']

all_predictions = [good_pred, wrong_pred]


for prediction in all_predictions:

    vector = 0
    i = 0
    
    for sentence in prediction:
        for word in sentence.split(","):

            word = word.replace("'", "")
            word = word.replace(" ", "")
            word = word.lower()

    #         if word is stopword, skip the word
            if word in stopword_list:
                continue

            word = stemmer.stem(word)

            try:

                vector += wv_twitter_50d[word]
                i += 1

            except:
                None
                
    vector = vector / i
    vectors.append(vector)


print(vectors)


[array([-8.92093033e-02, -2.45292753e-01,  8.20635110e-02, -3.35994102e-02,
       -8.69635418e-02, -8.36643483e-03,  5.47086298e-01,  1.65184528e-01,
        2.08654866e-01,  3.48535359e-01, -9.35202986e-02,  1.79868266e-01,
       -3.28301764e+00,  2.66943052e-02, -1.21933129e-02,  1.85036898e-01,
        9.95144770e-02, -3.34673285e-01, -1.51989043e-01, -1.23649687e-01,
       -2.20028803e-01,  8.54320708e-04,  1.69558764e-01, -1.01894453e-01,
        1.08210780e-02,  2.18606386e-02, -1.87982783e-01, -6.04968891e-02,
        8.27350989e-02, -1.98729858e-01,  4.09521982e-02,  3.18558961e-01,
        6.50063828e-02, -3.03752840e-01,  3.01999271e-01, -2.26621963e-02,
       -9.62538645e-03,  2.74224356e-02,  6.34580925e-02, -1.15649833e-03,
       -8.75078917e-01,  9.59849805e-02, -1.70730188e-01,  1.04005583e-01,
        2.70360529e-01, -1.33330375e-02,  3.16587865e-01,  3.78532968e-02,
       -8.57795961e-03,  2.58686971e-02], dtype=float32), array([-7.7965915e-02, -2.4658704e-01,  8

In [271]:
a = vectors[0].reshape(-1, 1)
b = vectors[1].reshape(-1, 1)
# print(a.shape)
a = a.mean()
b = b.mean()

# print(type(a))

# a = np.array([1, 4, 5, 765432, 5, 3])
# a = a.reshape(-1, 1)

pca = PCA(n_components=0.50, whiten = True)
result = pca.fit_transform(a)
result

# result
# pca = PCA(n_components=2)
# # pca.fit(vectors)
# # PCA(n_components=2)


# result = pca.fit(vectors)

# pca.explained_variance_ratio

ValueError: Expected 2D array, got scalar array instead:
array=-0.05679996311664581.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [251]:
a = [1, 2, 20000]
b = [4, 2, 5434543]

pca = PCA(n_components=2)
pca.fit_transform(a,b )

ValueError: Expected 2D array, got 1D array instead:
array=[1.e+00 2.e+00 2.e+04].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## POS tags distribution


In [72]:
from collections import Counter
import nltk
import pickle
from nltk import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')

with open('oov_dictionary.pickle', 'rb') as f:
    oov_dict = pickle.load(f)

top_10 = ['NOUN', 'DET', 'VERB', 'ADP', 'ADJ', '.', 'ADV', 'NUM', 'PRON', 'CONJ']
top_4 = ['NOUN', 'VERB', 'ADJ', 'ADV']

''' explanation:

ADJ	adjective	new, good, high, special, big, local
ADP	adposition	on, of, at, with, by, into, under
ADV	adverb	really, already, still, early, now
CONJ	conjunction	and, or, but, if, while, although
DET	determiner, article	the, a, some, most, every, no, which
NOUN	noun	year, home, costs, time, Africa
NUM	numeral	twenty-four, fourth, 1991, 14:24
PRT	particle	at, on, out, over per, that, up, with
PRON	pronoun	he, their, her, its, my, I, us
VERB	verb	is, say, told, given, playing, would
.	punctuation marks	. , ; !
X	other	ersatz, esprit, dunno, gr8, univeristy

'''

oov_dict



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


{'grey': 'gray',
 'ok.': 'OK',
 't-shirt': 'tshirt',
 'w/': 'with',
 'frige': 'fridge',
 'red/white': 'red and white',
 'doughnuts': 'donuts',
 'didnt': "didn't",
 'ok..': 'ok',
 'dr.': 'dr',
 'no..': 'no',
 'doughnut': 'donut',
 'yes..': 'yes',
 'yellow/orange': 'yellow and orange',
 'pic..': 'picture',
 'umberlla': 'umbrella',
 'cest': 'chest',
 'k..next': 'ok, next',
 'surboard': 'surfboard',
 'blue/white': 'blue and white',
 'man/woman': 'man or woman',
 'doesnt': "doesn't",
 'scheppa': 'schepps',
 'guy/girl': 'guy or girl',
 'blue/green': 'blue and green',
 'schepp': 'schepps',
 'blue/black': 'blue and black',
 'motarcycle': 'motorcycle',
 'red/black': 'red and black',
 'red/white/blue': 'red, white and blue',
 'colour': 'color',
 'close-up': 'closeup',
 'green/yellow': 'green and yellow',
 'red/pink': 'red and pink',
 'buddist': 'buddhist',
 'longsleeves': 'long sleeves',
 'ihave': 'i have',
 'yellow/green': 'yellow and green',
 'no-': 'no',
 'k.': 'ok',
 'surfobard': 'surfboard'

In [75]:
# text = " Guru99 is one of the best sites to learn WEB, SAP, Ethical Hacking and much more online."
messages = ['ik ben bob en daar is het strand maar toen zei hij']
pos_tags = []

for message in messages:   
    m_tokens = [i for i in word_tokenize(message.lower())]
    
    
    m_tokens_spell = []
    for word in m_tokens:
        if word not in oov_dict:
            m_tokens_spell.append(word)
        else:
            m_tokens_spell.append(oov_dict[word])
            
    pos_tags += [item[1] for item in pos_tag(m_tokens_spell, target='universal')]

# if len(pos_tags) == 0:
#     continue
pos_tag_dict = dict()       
for key,value in dict(Counter(pos_tags)).items():
    pos_tag_dict[key] = [value]  
    

# pos_tag_dict['Round_Nr'] = round_nr
# round_entry_df = pd.DataFrame.from_dict(pos_tag_dict)
# pos_statistics.append(round_entry_df)

TypeError: pos_tag() got an unexpected keyword argument 'target'